In [36]:
# Data based on:
# https://doi.org/10.1016/j.ymben.2014.07.004
# Libraries for the Dynamic block
import numpy as np
from scipy.integrate import odeint
# Libraries for the Metabolic block
import cobra
model=cobra.io.read_sbml_model("iMM904.xml")
# Plotting libraries
import matplotlib.pyplot as plt
from scipy.optimize import basinhopping

# Load data from xls

In [37]:
import pandas as pd
# En el terminal ejecutar 
# pip3 install openpyxl
data_s3=pd.read_excel(open('Datos_seccion_3.xlsx', 'rb'), sheet_name='200701010052')  
time_exp=data_s3["Time (h)"][10:1500]
do_exp=data_s3["DO"][10:1500]

idx = np.round(np.linspace(10, len(od_exp) - 1, 199)).astype(int)
od_exp_ss= od_exp[idx] # subset de 199 elementos tomados desde los 1490 elementos originales

# DFBA basado en el consumo observado de O2



<img src="https://raw.githubusercontent.com/mrivas/ModelamientoDeProcesosBiotecnologicos/master/2022/IMG_20220630_170141.jpg" width="55%" align="left" />

In [45]:
# Kinetic block
def kineticBlock(Ox,guess):
    # Parameters   
    vOxMax, K_Ox = guess 
    v_Ox=vOxMax*Ox/(K_Ox+Ox)

    return v_Ox

# Metabolic block
def metabolicBlock(v_Ox,Glu):
    if abs(v_Ox)>1e-4:  
        model.reactions.get_by_id("EX_o2_e").bounds = (-v_Ox,-v_Ox) # set flux of oxygen

        solution = model.optimize()
        u = solution.objective_value
        v_Glu = model.reactions.get_by_id("EX_glc__D_e").x   
    else:
        u, v_Glu,v_Ox = 0,0,0
    
    if Glu < 1e-4: 
        v_Glu=0
        u=0
    return [u, v_Glu,v_Ox]
    
# Dynamic block
def f(y,t,params):
    V,VX,VGlu,VOx = y # Current values
    F,u,v_Glu, v_Ox = params  # unpack parameters

    MW_Glu,MW_Ox = [0.18,.032] #Molecular weights 
    derivs=[F,                  # dV/dt
            u*VX,                # dVX/dt
            v_Glu*MW_Glu*(VX), # dVGlu/dt
            -v_Ox*MW_Ox*(VX)] # dVOx/dt
    return derivs

def dynamicBlock(y,params, ti,tf):
    time=np.linspace(ti,tf,100)
    #F,u,v_Glu, v_Eth, v_Gly, v_Cit, v_Lac = params
    soln = odeint(f,y,time,args=(params,))
    # Get solutions at the final time point (tf):
    V=soln[-1,0]
    X,Glu,Ox=soln[-1,1:4]/V
    return [V,X,Glu,Ox]    

# Save results along the fermentation path

def savePath(u,V,X,Glu,Ox,v_Glu,v_Ox):
    global u_path,V_path,X_path
    global Glu_path,Ox_path
    global v_Ox_path, v_Glu_path
    u_path += [u]
    V_path += [V]
    X_path += [X]
    Glu_path += [Glu]
    Ox_path += [Ox]
    v_Glu_path += [v_Glu]    
    v_Ox_path += [v_Ox]

In [48]:
#guess = 22.5/4,0.88
def predictor(guess):
    # Initial conditions
    Glu=20
    Ox=7.6*.94 #g/L
    V,X=[0.5,0.5]
    F=0.0

    # Running the simulation over time
    time=np.linspace(0,12,200)
    for i in range(len(time)):
        # KINETIC BLOCK: 
        # given the current concentrations of glucose (Glu) and ethanol (Eth)
        # we compute the fluxes of Glu, and upper and lower limits of Eth, Glycerol(Gly), Citrate(Cit), and Lactate(Lac)
        v_Ox = kineticBlock(Ox,guess)
        # METABOLIC BLOCK
        # given v_Glu,LB_Eth,LB_Gly,LB_Cit,LB_Lac
        # we compute biomass growth rate (u) and metabolic fluxes of extracellular metabolites
        u, v_Glu,v_Ox = metabolicBlock(v_Ox,Glu)
        # DYNAMIC BLOCK
        # given u, V, fluxes (v_Eth, v_Gly, v_Cit, v_Lac), and concentrations (X,Glu,Eth,Gly,Cit,Lac)
        # we update reaction volume V, X and Glu,Eth,Gly,Cit,Lac
        if i==len(time)-1: continue
        y = [V,X*V,Glu*V,Ox*V]
        params = [F,u,v_Glu, v_Ox]
        V,X,Glu,Ox = dynamicBlock(y, params, time[i],time[i+1])
        # Save results along the fermentaion path
        savePath(u,V,X,Glu,Ox,v_Glu,v_Ox)
    
    return(Ox_path)

In [49]:
# Una manera de medir la diferencia o error entre dos curvas es el Mean Square Error (MSE)

def mse(guess): # 
    # Datos predichos usando nuestra funcion
    predichos = predictor(guess) # regresa el Ox_path
    # Datos experimentales desde el laboratorio
    medidos=od_exp_ss
    # Calculas el error usando MSE
    error = predichos-medidos # diferencia entre predicho y medido
    error_square = np.square(error) # cada diferencia al cuadrado
    mse = np.mean(error_square) # promedio de las diferencias al cuadrado (MSE)
    return(mse)   


guess = [22.5/4,0.88]
minimizer_kwargs = {"method": "BFGS"}
sol = basinhopping(mse, guess, minimizer_kwargs=minimizer_kwargs,niter=1000,disp=False,stepsize=5)

UnboundLocalError: local variable 'u_path' referenced before assignment

In [8]:
guess = 22.5/4,0.88
u_path,V_path,X_path = [],[],[]
Glu_path,Ox_path=[],[]
v_Ox_path, v_Glu_path = [],[]
predictor(guess)

[7.139112075171438,
 7.1341108012600225,
 7.128993579385452,
 7.123757735241617,
 7.118400533458176,
 7.112919176287804,
 7.107310802226792,
 7.101572484602467,
 7.0957012301295315,
 7.089693977436376,
 7.0835475955605816,
 7.077258882413697,
 7.070824563214336,
 7.064241288889479,
 7.0575056344434275,
 7.050614097293707,
 7.0435630955737025,
 7.0363489664013725,
 7.028967964113594,
 7.021416258465701,
 7.013689932795678,
 7.005784982152609,
 6.997697311388873,
 6.989422733215699,
 6.980956966221597,
 6.972295632853303,
 6.963434257358789,
 6.954368263692005,
 6.945092973378956,
 6.935603603344817,
 6.925895263701734,
 6.915962955497059,
 6.905801568421762,
 6.895405878478813,
 6.884770545611318,
 6.873890111290346,
 6.862758996062276,
 6.851371497055685,
 6.839721785447766,
 6.827803903890346,
 6.815611763895665,
 6.803139143182112,
 6.790379682980213,
 6.777326885299235,
 6.763974110154876,
 6.750314572758615,
 6.7363413406693855,
 6.722047330908349,
 6.707425307037709,
 6.6924678762

# Tarea
Ubicar los datos predichos lo mas cercano posible a los datos medidos y entre ellos medir el error (MSE).

# Contenidos

1. Definir fecha de presentación final (corresponde también a la evaluación 2). Martes de 14 a 15:30pm, y desde las 15:30 a las 16:30 horas el Focus Group.
2. Responder dudas.
3. Revisar propuestas de código. Entre ellas una de Claudio.

199